In [1]:
import pandas as pd
import numpy as np
import torch
import torch.functional as F
import torch.nn as nn 
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, random_split, DataLoader, WeightedRandomSampler
from torchmetrics.classification import Accuracy, Precision
import os

### 1. Data Loading

In [22]:
df_path = 'Preprocessing/processed_df.parquet'
feature_path = 'Preprocessing/selected_features.parquet'

df = pd.read_parquet(df_path)
features = pd.read_parquet(feature_path)
    
y = df['Attrition'].values
selected_features = features[features['Final'] == 1].index
X = df.loc[:,selected_features].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)

### 2. Create Dataset

In [30]:
class myDataSet(Dataset):
    def __init__(self, data, label):
        super().__init__()
        self.data = torch.tensor(data, dtype=torch.float32)
        self.label = torch.tensor(label, dtype=torch.long)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.label[index]

### 3. Create Neural Network

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.BatchNorm1d(32),       
            nn.LeakyReLU(0.1),
            
            nn.Linear(32, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.1),
            
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),          

            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),

            nn.Linear(256, output_dim),
            nn.LogSoftmax(dim=1)      
        )
    
    def forward(self, x):
        return self.net(x)


#### 4. Parameters & Hyperparameters

In [52]:
# Parameter
IN_DIM = X.shape[1]
OUT_DIM = 2

# Hyperparameters
LEARNING_RATE = 1e-4
BATCH_SIZE = 16
NUM_EPOCHS = 100

In [31]:
train_data = myDataSet(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

valid_data = myDataSet(X_valid, y_valid)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)

test_data = myDataSet(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork(input_dim=IN_DIM, output_dim=OUT_DIM).to(device)

class_counts = np.bincount(y_train)  
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float32)
class_weights = class_weights / class_weights.sum() 
class_weights = class_weights.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)  
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
accuracy = Accuracy(task='multiclass', num_classes=OUT_DIM).to(device)
precision = Precision(task='multiclass', average='macro', num_classes=OUT_DIM).to(device)

best_acc = 0.0
best_prec = 0.0
best_model_path = "best_model.pth"

def train():
    global best_acc, best_prec

    for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            preds = model(x)
            
            loss = criterion(preds, y)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()
        acc_score = 0
        prec_score = 0
        
        with torch.no_grad():
            for x, y in valid_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x)
                pred_labels = preds.argmax(dim=1)

                acc_score += accuracy(pred_labels, y)
                prec_score += precision(pred_labels, y)
        
        acc_score /= len(valid_loader)
        prec_score /= len(valid_loader)

        print(f"Epoch {epoch+1}: Loss = {total_loss/len(train_loader):.4f}, Accuracy = {acc_score:.4f}, Precision = {prec_score:.4f}")

        if acc_score > best_acc:
            best_acc = acc_score
            torch.save(model.state_dict(), best_model_path)
            print(f"Best accuracy model saved at epoch {epoch+1} with Accuracy: {best_acc:.4f}, Precision: {best_prec:.4f}")
            
        if prec_score > best_prec:
            best_prec = prec_score
            torch.save(model.state_dict(), best_model_path)
            print(f"Best precision model saved at epoch {epoch+1} with Accuracy: {best_acc:.4f}, Precision: {best_prec:.4f}")

    torch.save(model.state_dict(), "final_model.pth")
    print(f"Training finished. Best Accuracy: {best_acc:.4f}, Best Precision: {best_prec:.4f}")


In [56]:
train()

Epoch 1: Loss = 0.6911, Accuracy = 0.6167, Precision = 0.5645
Best accuracy model saved at epoch 1 with Accuracy: 0.6167, Precision: 0.0000
Best precision model saved at epoch 1 with Accuracy: 0.6167, Precision: 0.5645
Epoch 2: Loss = 0.6507, Accuracy = 0.6604, Precision = 0.6064
Best accuracy model saved at epoch 2 with Accuracy: 0.6604, Precision: 0.5645
Best precision model saved at epoch 2 with Accuracy: 0.6604, Precision: 0.6064
Epoch 3: Loss = 0.6429, Accuracy = 0.6604, Precision = 0.6023
Epoch 4: Loss = 0.6454, Accuracy = 0.7167, Precision = 0.6250
Best accuracy model saved at epoch 4 with Accuracy: 0.7167, Precision: 0.6064
Best precision model saved at epoch 4 with Accuracy: 0.7167, Precision: 0.6250
Epoch 5: Loss = 0.6397, Accuracy = 0.6917, Precision = 0.5991
Epoch 6: Loss = 0.6239, Accuracy = 0.6854, Precision = 0.6088
Epoch 7: Loss = 0.6302, Accuracy = 0.7229, Precision = 0.6320
Best accuracy model saved at epoch 7 with Accuracy: 0.7229, Precision: 0.6250
Best precision mo